<a href="https://colab.research.google.com/github/lindseyvanosky/Pipelines-Activity-Exercise/blob/main/LV_Pipelines_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pipelines Activity
- Lindsey Vanosky 
- 06.23.22

##Import, Upload & Inspect

In [ ]:
# imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')


filename = "/content/drive/MyDrive/Coding Dojo/02 Stack 2: Intro to Machine Learning/Data Sets/Cereal with missing values.xlsx - Sheet 1 - cereal.csv"
df = pd.read_csv(filename)
df.head()


,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110.0,2,2.0,180.0,1.5,10.5,10.0,70,25.0,1,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130.0,3,2.0,NaN,2.0,18.0,NaN,100,25.0,3,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,NaN,6,2.0,290.0,2.0,17.0,1.0,105,25.0,1,1.00,1.25,50.764999
3,Cinnamon Toast Crunch,General Mills,Cold,120.0,1,3.0,210.0,0.0,13.0,9.0,45,25.0,2,1.00,0.75,19.823573
4,Clusters,General Mills,Cold,110.0,3,2.0,140.0,2.0,13.0,7.0,105,25.0,3,1.00,0.50,40.400208


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   name                                             77 non-null     object 
 1   Manufacturer                                     77 non-null     object 
 2   type                                             68 non-null     object 
 3   calories per serving                             70 non-null     float64
 4   grams of protein                                 77 non-null     int64  
 5   grams of fat                                     69 non-null     float64
 6   milligrams of sodium                             76 non-null     float64
 7   grams of dietary fiber                           77 non-null     float64
 8   grams of complex carbohydrates                   77 non-null     float64
 9   grams of sugars                   

In [ ]:
df.drop(columns = ["grams of protein", "name", "milligrams of sodium", "grams of dietary fiber", "grams of complex carbohydrates", "milligrams of potassium", "vitamins and minerals (% of FDA recommendation)", "Display shelf", "Number of cups in one serving", "Rating of cereal"], inplace = True)

In [ ]:
df.isna().sum()

Manufacturer                        0
type                                9
calories per serving                7
grams of fat                        8
grams of sugars                     9
Weight in ounces per one serving    0
dtype: int64

In [ ]:
df.dropna(inplace = True)

##Define Features & Validation Split 

In [ ]:
y = df["calories per serving"]
x = df.drop(columns = "calories per serving")

X_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)


The feature types are as followed:

Nominal: name, Manufacturer, type

Numerical: grams of fat, grams of sugars, Weight in ounces per one serving


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 76
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Manufacturer                      52 non-null     object 
 1   type                              52 non-null     object 
 2   calories per serving              52 non-null     float64
 3   grams of fat                      52 non-null     float64
 4   grams of sugars                   52 non-null     float64
 5   Weight in ounces per one serving  52 non-null     float64
dtypes: float64(4), object(2)
memory usage: 2.8+ KB


##Imputation

In [ ]:
df.isna().sum()

Manufacturer                        0
type                                0
calories per serving                0
grams of fat                        0
grams of sugars                     0
Weight in ounces per one serving    0
dtype: int64

In [ ]:
df["type"].value_counts()

Cold    50
Hot      2
Name: type, dtype: int64

In [ ]:
df["grams of fat"].value_counts()

0.0    21
1.0    19
2.0     8
3.0     3
5.0     1
Name: grams of fat, dtype: int64

In [ ]:
df["grams of sugars"].value_counts()

 3.0     8
 0.0     6
 6.0     5
 8.0     4
 12.0    4
 10.0    3
 7.0     3
 13.0    3
 11.0    3
 5.0     3
 9.0     2
 14.0    2
 2.0     2
 15.0    2
 4.0     1
-1.0     1
Name: grams of sugars, dtype: int64

###Building column selectors

In [ ]:
cat_selector = make_column_selector(dtype_include="object")
num_selector = make_column_selector(dtype_include="number")

###Building imputers

In [ ]:
freq_imputer = SimpleImputer(strategy = "most_frequent")
mean_imputer = SimpleImputer(strategy = "mean")

###OneHotEncode & Scaler

In [ ]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse = False)
scaler = StandardScaler()

###Pipelines

In [ ]:
numeric_pipeline = make_pipeline(mean_imputer, scaler)
cat_pipeline = make_pipeline(freq_imputer, ohe)

##Column Transform

In [ ]:
num_tuple = (numeric_pipeline, num_selector)
cat_tuple = (cat_pipeline, cat_selector)
preprocessor = make_column_transformer(num_tuple, cat_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fd2045a7810>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fd2045a7290>)])

###Fitting on train data

In [ ]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fd2045a7810>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fd2045a7290>)])

In [ ]:
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(x_test)

###Inspection

In [ ]:
print(np.isnan(x_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(x_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', x_train_processed.dtype)
print('All data in X_test_processed are', x_test_processed.dtype)
print('\n')
print('shape of data is', x_train_processed.shape)
print('\n')
x_train_processed



0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (39, 11)




array([[-0.04533761,  0.60508726, -0.23626638,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [-0.92942091, -0.54046629, -0.23626638,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [-0.04533761,  0.60508726, -0.23626638,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [-0.92942091, -0.99868771, -0.23626638,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.8387457 ,  1.29241939,  2.85581036,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [-0.04533761, -0.31135558, -0.23626638,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.   